# 미세먼지_db 종료 후 DF 작업하기
(여기서부터 시작)

In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
# from sqlalchemy import create_engine
# !pip install scikit-learn
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import skew, kurtosis
import os

In [ ]:
# matplotlib 한글 사용

plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

### 데이터 불러와서 가공하기

In [ ]:
fine_dust_df = pd.read_csv('./data/fine_dust.csv')
fine_dust_df.columns

In [ ]:
# 필요 컬럼만 자르기
fine_dust = fine_dust_df.loc[:, ['date', 'fine_dust(㎍/㎥)']] # 잘랐으면 비활성화 안 잘랐으면 활성화
fine_dust.info()

미세먼지 등급별 값 메기기

In [ ]:
# 좋음 (0-30 ㎍/m³) → Good
# 보통 (31-80 ㎍/m³) → Moderate
# 나쁨 (81-150 ㎍/m³) → Bad
# 매우 나쁨 (151 ㎍/m³ 이상) → Very Bad

# 새로운 컬럼을 생성하고 조건에 따라 값 입력

conditionlist = [
    (fine_dust['fine_dust(㎍/㎥)'] <= 30),
    (fine_dust['fine_dust(㎍/㎥)'] > 30) & (fine_dust['fine_dust(㎍/㎥)'] <= 80),
    (fine_dust['fine_dust(㎍/㎥)'] > 80) & (fine_dust['fine_dust(㎍/㎥)'] <= 150),
    (fine_dust['fine_dust(㎍/㎥)'] > 150),
]

values = ['good', 'normal', 'bad', 'very_bad']

fine_dust["grade"] = np.select(conditionlist, values, default="unknown")
fine_dust.head(2)

fine_dust의 date의 타입 변경 후 평일/주말 나누기

In [ ]:
fine_dust.info() # date 타입이 object임을 확인
fine_dust['date'] = pd.to_datetime(fine_dust['date'])
# fine_dust.info() # date타입이 datetime64[ns]으로 변경됨

In [ ]:
# fine_dust["date"]에 따라 평일, 주말 나누기(월~금 : 0~4, 토일 : 5, 6)
fine_dust["day"] = np.where(fine_dust["date"].dt.weekday < 5, "weekday", "weekend")
fine_dust


공휴일 데이터인 hd의 date의 타입 변경

In [ ]:
hd = pd.read_csv('./data/hd_21_24.csv', encoding="CP949")
hd.rename(columns={"일시":"date"}, inplace=True)
hd['date'] = pd.to_datetime(hd['date'])
hd.info()

fine_dust['day_type'] 만들어서 평일/주말+공휴일로 나누기 (공휴일은 주말로 처리)

In [ ]:
# fine_dust와 hd DF 합치기
m_fine_dust = pd.merge(fine_dust, hd, how='outer', on='date')
m_fine_dust.head()

In [ ]:
# apply 활용
date_name2 = m_fine_dust.apply(lambda row : 'weekend' 
                               if str(row['dateName']) != 'nan' else row['day'], axis=1)

In [ ]:
# apply 활용해서 평일, 주말(+공휴일)로 나눈 값을
# 합친 df에 적용하기 
m_fine_dust['day_type'] = date_name2

# 필요한 컬럼/행으로 자르기
m_fine_dust = m_fine_dust.loc[:, ['date', 'fine_dust(㎍/㎥)', 'grade', 'day_type']]
m_fine_dust = m_fine_dust.iloc[:1329, :]
m_fine_dust.head(1), m_fine_dust.tail(1)

병합된 DF을 계절별, 연도별로 구분하기

In [ ]:
# date를 기준으로 계절별 구분 (봄: 3~5, 여름: 6~8, 가을: 9~11, 겨울: 12~2)
conditions = [
    ( m_fine_dust["date"].dt.month >= 3 ) & ( m_fine_dust["date"].dt.month <= 5 ),
    ( m_fine_dust["date"].dt.month >= 6 ) & ( m_fine_dust["date"].dt.month <= 8 ),
    ( m_fine_dust["date"].dt.month >= 9 ) & ( m_fine_dust["date"].dt.month <= 11 ),
]
season = ["Spring", "Summer", "Fall"]
m_fine_dust["season"] = np.select(conditions, season, default="Winter")

print( m_fine_dust.iloc[57:59, :] )
print( m_fine_dust.iloc[145:147, :] )
print( m_fine_dust.iloc[237:239, :] )
print( m_fine_dust.iloc[325:327, :] )

In [ ]:
# date를 기준으로 연도 구분
m_fine_dust["year"] = m_fine_dust["date"].dt.year
print( m_fine_dust.iloc[356:358, :] )

결측치 수정 ( 미세먼지 농도 결측치를 포함되는 계절의 평균으로 대체 )

In [ ]:
print( "평일 수 : ", len(m_fine_dust.loc[m_fine_dust["day_type"] == "weekday"]) )
print( "주말 및 공휴일 수 : ", len(m_fine_dust.loc[m_fine_dust["day_type"] == "weekend"]) )

In [ ]:
# m_fine_dust[m_fine_dust['fine_dust(㎍/㎥)'].isnull()]
# 결측치 행
        # 58	2021-03-01	NaN	NaN	weekend
        # 504	2022-06-06	NaN	NaN	weekend
        # 620	2022-10-10	NaN	NaN	weekend
        # 822	2023-05-05	NaN	NaN	weekend
        # 1176	2024-05-15	NaN	NaN	weekend
        # 1293	2024-09-16	NaN	NaN	weekend
# 계절별 평균을 결측치에 넣기로 함

# 계절별 평균 구하기
season_means = m_fine_dust.groupby('season')['fine_dust(㎍/㎥)'].mean()
                # Fall      28.840909
                # Spring    57.397101
                # Summer    28.062500
                # Winter    46.641509

# 결측치행에 평균값 넣기
m_fine_dust["fine_dust(㎍/㎥)"] = m_fine_dust["fine_dust(㎍/㎥)"]. \
                        fillna(m_fine_dust.groupby("season")["fine_dust(㎍/㎥)"].transform("mean"))

m_fine_dust[m_fine_dust['fine_dust(㎍/㎥)'].isnull()]


범주화

In [ ]:
# 범주화
# 순서있는 범주형 : ordered = True, 순서 없는 범주형 : ordered = False
c_grade = ['good', 'normal', 'bad', 'very_bad']
c_grade1 = CategoricalDtype(categories = c_grade, ordered = True)

c_season = ["Spring", "Summer", "Fall", "Winter"]
c_season1 = CategoricalDtype(categories = c_season, ordered = True)

m_fine_dust["grade"] = m_fine_dust["grade"].astype(c_grade1)
m_fine_dust["season"] = m_fine_dust["season"].astype(c_season1)
m_fine_dust.info()

### 데이터 분석

1. 단일 그룹 분석

In [ ]:
def group_analysis_by_single_column(df, group_col_name, col_name):
    # 그룹화
    grouped_df = df.groupby(group_col_name, observed=True)

    # 그룹별 최대/최소/평균/분산/빈도 계산
    stats = grouped_df[col_name].agg([
        ("Max", "max"),
        ("Min", "min"),
        ("Mean", "mean"),
        ("Variance", "var"),
        ("Count", "count")
    ])
    return stats

# 단일 그룹 분석 결과를 데이터 프레임으로 저장
df_year_stats = group_analysis_by_single_column(
                    df = m_fine_dust, 
                    group_col_name = "year", 
                    col_name = "fine_dust(㎍/㎥)")
df_grade_stats = group_analysis_by_single_column(
                    df = m_fine_dust, 
                    group_col_name = "grade", 
                    col_name = "fine_dust(㎍/㎥)")
df_season_stats = group_analysis_by_single_column(
                    df = m_fine_dust, 
                    group_col_name = "season", 
                    col_name = "fine_dust(㎍/㎥)")
df_day_stats = group_analysis_by_single_column(
                    df = m_fine_dust, 
                    group_col_name = "day_type", 
                    col_name = "fine_dust(㎍/㎥)")

# 딕셔너리로 데이터프레입 합치기
combined_stats = {
    "Year": df_year_stats,
    "Grade": df_grade_stats,
    "Season": df_season_stats,
    "Day Type": df_day_stats
}

print( "단일 그룹 분석")
# 결과 확인
for key, value in combined_stats.items():
    print(f"\n\n{key} Stats:\n", value)


그래프별 데이터 분석 (분석내용 주석처리)

<!-- 
Yearly Fine Dust Statistics (연도별 미세먼지 통계):
2021년이 가장 높은 최대값(약 380)
2022-2023년은 최대값이 감소하는 추세
2024년(1월-10월30일)은 다른 연도보다 데이터 수(count)가 적음
전반적으로 평균값은 50 미만으로 유지됨

Grade-wise Fine Dust Statistics (등급별 미세먼지 통계):
very_bad(매우 나쁨) 등급이 가장 높은 최대값(약 380)
등급이 나빠질수록(good→very_bad) 최대값, 최소값, 평균값 모두 증가
good(좋음)과 normal(보통)은 상대적으로 많은 발생 빈도
very_bad는 발생 빈도가 가장 낮음

Seasonal Fine Dust Statistics (계절별 미세먼지 통계):
봄(Spring)에 가장 높은 최대값
여름(Summer)과 가을(Fall)은 상대적으로 낮은 수치
봄과 여름이 다른 계절보다 발생 빈도가 높음
겨울(Winter)은 중간 정도의 수치를 보임

Weekday/Weekend Fine Dust Statistics (평일/주말 미세먼지 통계):
평일(weekday)과 주말(weekend)의 최대값이 비슷
평일이 주말보다 약간 더 많은 발생 빈도
평균값은 평일과 주말이 비슷한 수준(약 40)

전반적인 특징:
계절성: 봄철에 미세먼지가 가장 심각
연도별 개선: 2021년 이후 최대값이 감소하는 추세
발생 빈도: 좋음/보통 등급이 가장 많이 발생
평일/주말 차이: 큰 차이는 없으나 평일이 조금 더 빈번

미세먼지 관리나 대책 수립 시 봄철에 더 집중적인 관리가 필요하며, 
전반적으로 미세먼지 상황이 개선되고 있다는 것을 알 수 있음
 -->

2. 두 개 변수 조합 분석

In [ ]:
# 2-1. 연도와 등급을 조합해서 분석
# 2-2. 연도와 계절을 조합해서 분석
# 2-3. 연도와 평일/주말을 조합해서 분석
# 2-4. 등급과 계절을 조합해서 분석
# 2-5. 등급과 평일/주말을 조합해서 분석
# 2-6. 계절과 평일/주말을 조합해서 분석

In [ ]:
def group_analysis_by_two_columns(df, group_col1, group_col2, target_col):
    # 두 컬럼으로 그룹화
    grouped_df = df.groupby([group_col1, group_col2], observed=True)
    
    # 통계 계산 및 컬럼 이름 지정 (열 이름, 계산 방법)
    stats = grouped_df[target_col].agg([
    ("Max", "max"),
    ("Min", "min"),
    ("Mean", "mean"),
    ("Variance", "var"),
    ("Count", "count"),
    ("Median", "median"),                   # 중앙값
    ("Q1", lambda x: x.quantile(0.25)),     # 1사분위수 (Q1)
    ("Q3", lambda x: x.quantile(0.75)),     # 3사분위수 (Q3)
    ("IQR", lambda x: x.quantile(0.75) - x.quantile(0.25)),     # IQR
    ("Skewness", lambda x: skew(x)),        # 왜도
    ("Kurtosis", lambda x: kurtosis(x)),    # 첨도
    ("Range", lambda x: x.max() - x.min())  # 범위
])
    return stats

# 함수 실행 예시
df_year_grade_stats = group_analysis_by_two_columns(
    df = m_fine_dust,
    group_col1 = "year", 
    group_col2 = "grade", 
    target_col = "fine_dust(㎍/㎥)")
df_year_season_stats = group_analysis_by_two_columns(
    df = m_fine_dust,
    group_col1 = "year", 
    group_col2 = "season", 
    target_col = "fine_dust(㎍/㎥)")
df_year_day_stats = group_analysis_by_two_columns(
    df = m_fine_dust,
    group_col1 = "year", 
    group_col2 = "day_type", 
    target_col = "fine_dust(㎍/㎥)")
df_grade_season_stats = group_analysis_by_two_columns(
    df = m_fine_dust,
    group_col1 = "grade", 
    group_col2 = "season", 
    target_col = "fine_dust(㎍/㎥)")
df_grade_day_stats = group_analysis_by_two_columns(
    df = m_fine_dust,
    group_col1 = "grade", 
    group_col2 = "day_type", 
    target_col = "fine_dust(㎍/㎥)")
df_season_day_stats = group_analysis_by_two_columns(
    df = m_fine_dust,
    group_col1 = "season", 
    group_col2 = "day_type", 
    target_col = "fine_dust(㎍/㎥)")

# 결과를 딕셔너리로 정리
double_group_stats = {
    "Year-Grade": df_year_grade_stats,
    "Year-Season": df_year_season_stats,
    "Year-Day": df_year_day_stats,
    "Grade-Season": df_grade_season_stats,
    "Grade-Day": df_grade_day_stats,
    "Season-Day": df_season_day_stats
}

# 결과 출력
print("2. 이중 그룹 분석")
for key, value in double_group_stats.items():
    print(f"\n{key} Stats:\n", value)

3. 세 개 변수 조합 분석

In [ ]:
# 3-1. 연도와 등급과 계절을 조합해서 분석
# 3-2. 연도와 등급과 평일/주말을 조합해서 분석
# 3-3. 등급과 계절과 평일/주말을 주합해서 분석

In [ ]:
def group_analysis_by_three_columns(df, group_col1, group_col2, group_col3, target_col):
    # 세 컬럼으로 그룹화
    grouped_df = df.groupby([group_col1, group_col2, group_col3], observed=True)

    # 통계 계산 및 컬럼 이름 지정
    stats = grouped_df[target_col].agg([
    ("Max", "max"),                    # (열 이름, 계산 방법)
    ("Min", "min"),
    ("Mean", "mean"),
    ("Variance", "var"),
    ("Count", "count"),
    ("Median", "median"),                   # 중앙값
    ("Q1", lambda x: x.quantile(0.25)),     # 1사분위수 (Q1)
    ("Q3", lambda x: x.quantile(0.75)),     # 3사분위수 (Q3)
    ("IQR", lambda x: x.quantile(0.75) - x.quantile(0.25)),     # IQR
    ("Skewness", lambda x: skew(x)),        # 왜도
    ("Kurtosis", lambda x: kurtosis(x)),    # 첨도
    ("Range", lambda x: x.max() - x.min())  # 범위
])
    return stats

# 함수 실행 예시
df_year_grade_season_stats = group_analysis_by_three_columns(
                                df = m_fine_dust,
                                group_col1 = "year", 
                                group_col2 = "grade", 
                                group_col3 = "season", 
                                target_col = "fine_dust(㎍/㎥)")
df_year_grade_day_stats = group_analysis_by_three_columns(
                                df = m_fine_dust,
                                group_col1 = "year", 
                                group_col2 = "grade", 
                                group_col3 = "day_type", 
                                target_col = "fine_dust(㎍/㎥)")
df_grade_season_day_stats = group_analysis_by_three_columns(
                                df = m_fine_dust,
                                group_col1 = "grade", 
                                group_col2 = "season", 
                                group_col3 = "day_type", 
                                target_col = "fine_dust(㎍/㎥)")

# 결과를 딕셔너리로 정리
triple_group_stats = {
    "Year-Grade-Season": df_year_grade_season_stats,
    "Year-Grade-Day": df_year_grade_day_stats,
    "Grade-Season-Day": df_grade_season_day_stats,
}

# 결과 출력
print("2. 삼중 그룹 분석")
for key, value in triple_group_stats.items():
    print(f"\n{key} Stats:\n", value)

4. 모든 변수 조합 분석

In [ ]:
# 4-1. 연도와 등급과 계절과 평일/주말을 조합해서 분석

In [ ]:
def group_analysis_by_four_columns(df, group_col1, group_col2, group_col3, group_col4, target_col):
    # 네 컬럼으로 그룹화
    grouped_df = df.groupby([group_col1, group_col2, group_col3, group_col4], observed=True)

    # 통계 계산 및 컬럼 이름 지정
    stats = grouped_df[target_col].agg([
    ("Max", "max"),                    # (열 이름, 계산 방법)
    ("Min", "min"),
    ("Mean", "mean"),
    ("Variance", "var"),
    ("Count", "count"),
    ("Median", "median"),                   # 중앙값
    ("Q1", lambda x: x.quantile(0.25)),     # 1사분위수 (Q1)
    ("Q3", lambda x: x.quantile(0.75)),     # 3사분위수 (Q3)
    ("IQR", lambda x: x.quantile(0.75) - x.quantile(0.25)),     # IQR
    ("Skewness", lambda x: skew(x)),        # 왜도
    ("Kurtosis", lambda x: kurtosis(x)),    # 첨도
    ("Range", lambda x: x.max() - x.min())  # 범위
])
    return stats

# 함수 실행 예시
df_year_season_grade_day_stats = group_analysis_by_four_columns(
                                df = m_fine_dust,
                                group_col1 = "year", 
                                group_col2 = "season",
                                group_col3 = "grade",  
                                group_col4 = "day_type",
                                target_col = "fine_dust(㎍/㎥)")

# 결과를 딕셔너리로 정리
quadruple_group_stats = {
    "Year-Grade-Season": df_year_season_grade_day_stats
}

# 결과 출력
print("2. 사중 그룹 분석")
for key, value in quadruple_group_stats.items():
    print(f"\n{key} Stats:\n", value)

딕셔너리를 csv파일로 저장하기

In [ ]:
import os
import pandas as pd

def save_stats_with_check(combined_stats, double_group_stats, triple_group_stats, quadruple_group_stats):
    print("데이터 구조 확인 중...")
    print_data_structure(combined_stats, "combined_stats")
    print_data_structure(double_group_stats, "double_group_stats")
    print_data_structure(triple_group_stats, "triple_group_stats")
    print_data_structure(quadruple_group_stats, "quadruple_group_stats")
    
    print("\n저장 시작...")
    save_multiindex_stats_to_csv(
        combined_stats,
        double_group_stats,
        triple_group_stats,
        quadruple_group_stats
    )

def print_data_structure(data, name):
    """데이터 구조를 출력하여 확인"""
    print(f"{name} 구조:")
    if isinstance(data, dict):
        print({key: type(value) for key, value in data.items()})
    else:
        print(f"{name} is {type(data)}")

def save_multiindex_stats_to_csv(*args):
    """각 딕셔너리 내 DataFrame을 개별 CSV로 저장"""
    # Set the directory path
    directory = "../dict_to_csv/"
    # Create the directory if it does not exist
    os.makedirs(directory, exist_ok=True)
    
    for i, stats in enumerate(args, start=1):
        if isinstance(stats, dict):
            for key, df in stats.items():
                if isinstance(df, pd.DataFrame):
                    filename = os.path.join(directory, f"stats_{i}_{key}.csv")
                    df.to_csv(filename)
                    print(f"{filename} 파일로 저장되었습니다.")
                else:
                    print(f"{key}는 DataFrame 형식이 아니므로 CSV로 저장할 수 없습니다.")
        else:
            print(f"stats_{i}는 딕셔너리 형식이 아닙니다.")

# 예제 실행 (실제 데이터로 교체)
save_stats_with_check(combined_stats, double_group_stats, triple_group_stats, quadruple_group_stats)


# 날씨_db 종료 후 DF 작업하기
(여기서부터 시작)

In [2]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
# from sqlalchemy import create_engine
# !pip install scikit-learn
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import skew, kurtosis
import os

In [3]:
# matplotlib 한글 사용

plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

### 데이터 불러와서 가공하기 - 강수량

In [4]:
wt_df = pd.read_csv('./data/weather_data.csv')
print(wt_df.columns)
wt_df.head(1)

Index(['Unnamed: 0', 'station_id', 'station_name', 'date_time', 'avg_temp',
       'min_temp', 'max_temp', 'daily_precipitation', 'avg_wind_speed',
       'daily_snow_depth'],
      dtype='object')


,Unnamed: 0,station_id,station_name,date_time,avg_temp,min_temp,max_temp,daily_precipitation,avg_wind_speed,daily_snow_depth
0,0,108,서울,2021-01-01,-4.2,-9.8,1.6,NaN,2.0,NaN


In [5]:
# 필요 컬럼만 자르기 - 강수량
wt_df = wt_df.loc[:, ['date_time', 'daily_precipitation']] 
wt_df.columns = ['Date', 'Precipitation(mm)']
wt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1399 entries, 0 to 1398
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               1399 non-null   object 
 1   Precipitation(mm)  591 non-null    float64
dtypes: float64(1), object(1)
memory usage: 22.0+ KB


In [6]:
wt_df["Precipitation(mm)"].max()

np.float64(176.2)

강수량별 값 메기기

In [7]:
# 건조 (Dry): 강수량이 없는 날, 즉 0mm
# 약강수 (Light): 0mm < 강수량 ≤ 10mm
# 중강수 (Moderate): 10mm < 강수량 ≤ 50mm
# 대강수 (Heavy): 강수량 > 50mm

# 새로운 컬럼을 생성하고 조건에 따라 값 입력

precipitation_list = [
    (wt_df['Precipitation(mm)'] == 00),
    (wt_df['Precipitation(mm)'] > 00) & (wt_df['Precipitation(mm)'] <= 10),
    (wt_df['Precipitation(mm)'] > 10) & (wt_df['Precipitation(mm)'] <= 50),
    (wt_df['Precipitation(mm)'] > 50),
]

precipitation_values = ['Dry', 'Light', 'Moderate', 'Heavy']

wt_df["Grade"] = np.select(precipitation_list, precipitation_values, default="Dry")
wt_df.head(2)

,Date,Precipitation(mm),Grade
0,2021-01-01,NaN,Dry
1,2021-01-02,NaN,Dry


wt_df의 Date의 타입 변경 후 평일/주말 나누기

In [ ]:
wt_df.info() # date 타입이 object임을 확인
wt_df['Date'] = pd.to_datetime(wt_df['Date'])
# wt_df.info() # date타입이 datetime64[ns]으로 변경됨

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1399 entries, 0 to 1398
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               1399 non-null   object 
 1   Precipitation(mm)  591 non-null    float64
 2   Grade              1399 non-null   object 
dtypes: float64(1), object(2)
memory usage: 32.9+ KB


In [9]:
# wt_df["Date"]에 따라 평일, 주말 나누기(월~금 : 0~4, 토일 : 5, 6)
wt_df["Day"] = np.where(wt_df["Date"].dt.weekday < 5, "Weekday", "Weekend")
wt_df


,Date,Precipitation(mm),Grade,Day
0,2021-01-01,NaN,Dry,Weekday
1,2021-01-02,NaN,Dry,Weekend
2,2021-01-03,NaN,Dry,Weekend
3,2021-01-04,0.0,Dry,Weekday
4,2021-01-05,0.0,Dry,Weekday
...,...,...,...,...
1394,2024-10-26,NaN,Dry,Weekend
1395,2024-10-27,0.0,Dry,Weekend
1396,2024-10-28,0.0,Dry,Weekday
1397,2024-10-29,0.0,Dry,Weekday


공휴일 데이터인 hd의 date의 타입 변경

In [10]:
hd = pd.read_csv('./data/hd_21_24.csv', encoding="CP949")
hd.rename(columns={"일시":"date"}, inplace=True)
hd['date'] = pd.to_datetime(hd['date'])
hd.columns=['Date', 'dateName']
hd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      66 non-null     datetime64[ns]
 1   dateName  66 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 1.2+ KB


fine_dust['Day_type'] 만들어서 평일/주말+공휴일로 나누기 (공휴일은 주말로 처리)

In [12]:
# fine_dust와 hd DF 합치기
wt_df = pd.merge(wt_df, hd, how='outer', on='Date')
wt_df.head()

,Date,Precipitation(mm),Grade,Day,dateName
0,2021-01-01,NaN,Dry,Weekday,1월1일
1,2021-01-02,NaN,Dry,Weekend,NaN
2,2021-01-03,NaN,Dry,Weekend,NaN
3,2021-01-04,0.0,Dry,Weekday,NaN
4,2021-01-05,0.0,Dry,Weekday,NaN


In [13]:
# apply 활용
date_name2 = wt_df.apply(lambda row : 'Weekend' 
                               if str(row['dateName']) != 'nan' else row['Day'], axis=1)

In [14]:
# apply 활용해서 평일, 주말(+공휴일)로 나눈 값을
# 합친 df에 적용하기 
wt_df['Day_type'] = date_name2

# 필요한 컬럼/행으로 자르기
wt_df = wt_df.loc[:, ['Date', 'Precipitation(mm)', 'Grade', 'Day_type']]
wt_df = wt_df.iloc[:1329, :]
wt_df.head(1), wt_df.tail(1)

(        Date  Precipitation(mm) Grade Day_type
 0 2021-01-01                NaN   Dry  Weekend,
            Date  Precipitation(mm)     Grade Day_type
 1328 2024-08-21               42.2  Moderate  Weekday)

병합된 DF을 계절별, 연도별로 구분하기

In [16]:
# date를 기준으로 계절별 구분 (봄: 3~5, 여름: 6~8, 가을: 9~11, 겨울: 12~2)
conditions = [
    ( wt_df["Date"].dt.month >= 3 ) & ( wt_df["Date"].dt.month <= 5 ),
    ( wt_df["Date"].dt.month >= 6 ) & ( wt_df["Date"].dt.month <= 8 ),
    ( wt_df["Date"].dt.month >= 9 ) & ( wt_df["Date"].dt.month <= 11 ),
]
Season = ["Spring", "Summer", "Fall"]
wt_df["Season"] = np.select(conditions, Season, default="Winter")

print( wt_df.iloc[57:59, :] )
print( wt_df.iloc[145:147, :] )
print( wt_df.iloc[237:239, :] )
print( wt_df.iloc[325:327, :] )

         Date  Precipitation(mm) Grade Day_type  Season
57 2021-02-27                NaN   Dry  Weekend  Winter
58 2021-02-28                NaN   Dry  Weekend  Winter
          Date  Precipitation(mm)  Grade Day_type  Season
145 2021-05-26                0.0    Dry  Weekday  Spring
146 2021-05-27                8.9  Light  Weekday  Spring
          Date  Precipitation(mm)  Grade Day_type  Season
237 2021-08-26                0.7  Light  Weekday  Summer
238 2021-08-27                0.9  Light  Weekday  Summer
          Date  Precipitation(mm)  Grade Day_type Season
325 2021-11-22                1.3  Light  Weekday   Fall
326 2021-11-23                NaN    Dry  Weekday   Fall


In [18]:
# date를 기준으로 연도 구분
wt_df["Year"] = wt_df["Date"].dt.year
print( wt_df.iloc[356:358, :] )
wt_df.info()

          Date  Precipitation(mm)  Grade Day_type  Season  Year
356 2021-12-23                NaN    Dry  Weekday  Winter  2021
357 2021-12-24                0.2  Light  Weekday  Winter  2021
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1329 entries, 0 to 1328
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               1329 non-null   datetime64[ns]
 1   Precipitation(mm)  559 non-null    float64       
 2   Grade              1329 non-null   object        
 3   Day_type           1329 non-null   object        
 4   Season             1329 non-null   object        
 5   Year               1329 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(1), object(3)
memory usage: 57.2+ KB


결측치 수정 ( 강수량 NaN을 0으로 바꿈 )

In [ ]:
wt_df["Precipitation(mm)"].fillna(0, inplace=True)
wt_df.head(2)

C:\Users\human\AppData\Local\Temp\ipykernel_8496\2003089569.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  wt_df["Precipitation(mm)"].fillna(0, inplace=True)


,Date,Precipitation(mm),Grade,Day_type,Season,Year
0,2021-01-01,0.0,Dry,Weekend,Winter,2021
1,2021-01-02,0.0,Dry,Weekend,Winter,2021
